# Working with MANTO data

<div class="alert alert-warning" style="margin: 1em 2em">
    <h4>Rate limit on JSON requests to MANTO server</h4>
    <p>Note that MANTO's API limits JSON requests to 1000 / 15 minutes from a given IP. This should be <strong>more than sufficient</strong> for use with DICES, since MANTO entities are cached locally after the first download. If, however, you do exceed the limit, then the call to MANTO will return an error and the DICES client will create a dummy MANTO entity with no data.</p>
    <p>I've introduced the <code>cache_empty=False</code> default for <code>getMantoID()</code> and <code>getMantoChar()</code>, which specifies that an entity returned with an empty <code>data</code> attribute should not be cached, forcing an automatic re-download the next time it's used. If, however, you expect <code>data</code> to be empty, you can set <code>cache_empty=True</code> to prevent unnecessary calls to the MANTO server.</p>
</div>

### Preliminaries

Import statemtents. Our MANTO tools are in the `dicesapi.manto` module.

In [1]:
from dicesapi import DicesAPI
from dicesapi.jupyter import NotebookPBar
from dicesapi import manto

Create a connection to DICES

In [2]:
api=DicesAPI(progress_class=NotebookPBar, logfile='dices-manto.log')

Instantiate a DICES character to get started

In [3]:
ach = api.getCharacters(name='Achilles')[0]
print(ach)

<Character 14: Achilles>


### Retrieving MANTO entities

Let’s create a new MANTO entity by retrieving Achilles’ ID from MANTO.

In [4]:
manto_ent = manto.getMantoChar(ach)
print(manto_ent)

<MANTO Entity 8182012: 👤 Achilles>


### Using MANTO ties to find relatives

This code is still in development and subject to change. But for now, this is how we search MANTO for Achilles' parents. Under the hood, we're checking two different MANTO ties, `31764`, "son of", and `31765`, "daughter of."

This function currently returns MANTO Entity objects.

In [5]:
parents = manto_ent.getParents()
print(parents)

[<MANTO Entity 8189146: 👤 Peleus>, <MANTO Entity 8190121: 👤 Thetis>]


Let's look for grandparents. Note that when sources disagree about parentage, MANTO returns an inclusive list.

In [6]:
for p in parents:
    print(f'Grandparents on {p.name}’s side:')
    for gp in p.getParents():
        print(' - ' + gp.name)

Grandparents on 👤 Peleus’s side:
 - 👤 Aiacos
 - 👤 Endeis
Grandparents on 👤 Thetis’s side:
 - 👤 Doris (Oceanid)
 - 👤 Oceanos (Titan)
 - 👤 Nereus


### Boolean relationship checker

#### For MANTO Entities

Sometimes we just want a yes/no answer as to parentage. For MANTO Entities, methods like `isChildOf()` will take a second Manto entity as argument and return `True` or `False`. This can be useful, for example, for filtering a list.

In [7]:
# a list of character pairs, by MANTO ID
pairs = [
    ('8182179', '8182084'),  # Anchises and Aeneas
    ('8188282', '8190182'),  # Eteocles and Oedipus
    ('8189802', '8182035'),  # Menelaus and Agamemnon
]

for a, b in pairs:
    ent_a = manto.getMantoID(a)
    ent_b = manto.getMantoID(b)
    print(f'Is {ent_a.name} the child of {ent_b.name}? ', ent_a.isChildOf(ent_b))

Is 👤 Anchises (Trojan) the child of 👤 Aineias?  False
Is 👤 Eteocles (Theban) the child of 👤 Oedipus?  True
Is 👤 Menelaos the child of 👤 Agamemnon?  False


#### For DICES Characters

For convenience, there are also wrapper functions that perform the same checks on DICES Characters by resolving the characters to MANTO entities first, then calling the boolean methods above. If you pass a CharacterInstance instead of a Character, the function will try to determine the MANTO ID of the underlying Character.

If passed a Character with no MANTO ID, or an anonymous CharacterInstance with no Character, these functions return `None`. You can use the optional `err_val` parameter to set a different fallback value; for example, if you're using the function in a filter and want to ensure it always returns a Boolean, set this to `False`.

For these next examples, let's start with a list of Achilles' interlocutors.

In [8]:
# download all speeches by Achilles
speeches = api.getSpeeches(spkr_name='Achilles', work_title='Iliad', progress=True)
print(f'{len(speeches)} speeches')

# extract the list of addressees
addressees = speeches.getAddrs()
print(f'{len(addressees)} unique addressees.')

88 speeches
30 unique addressees.


In this block, we use call `charIsChild()` on the speaker and his addressees to test parentage using MANTO's data. Where the function returns `None`, we know data was missing.

In [9]:
for addr in addressees:
    check = manto.charIsChild(ach, addr)
    if check is not None:
        print(f'Is {ach.name} the child of {addr.name}?', check)
    else:
        print(f'No MANTO data available for {addr.name}')

Is Achilles the child of Agamemnon? False
Is Achilles the child of Calchas? False
Is Achilles the child of Athena? False
Is Achilles the child of Eurybates? False
Is Achilles the child of Patroclus? False
Is Achilles the child of Talthybius? False
Is Achilles the child of Thetis? True
Is Achilles the child of Aias (son of Telamon)? False
Is Achilles the child of Odysseus? False
Can't get MANTO data: <Character 766: Phoinix> has no MANTO id
No MANTO data available for Phoinix
Can't get MANTO data: <CharacterInstance 85: Myrmidons> is an anonymous CharacterInstance
No MANTO data available for Myrmidons
Is Achilles the child of Zeus? False
Is Achilles the child of Achilles? False
Is Achilles the child of Iris? False
Can't get MANTO data: <CharacterInstance 3: Greeks> is an anonymous CharacterInstance
No MANTO data available for Greeks
Can't get MANTO data: <CharacterInstance 57: horses> is an anonymous CharacterInstance
No MANTO data available for horses
Is Achilles the child of Aeneas? F

But if we want to use the function in a custom filter, we can silence the errors and force it to just return `False` when it doesn't find MANTO data.

In [10]:
# filter a CharacterInstanceGroup based on MANTO ties:
parent_addrs = addressees.advancedFilter(lambda addr: manto.charIsChild(ach, addr, debug=False, err_val=False))

print('Parent addressees of Achilles:')
for p in parent_addrs:
    print(p.name)
print()
    
# now use the filtered list of CharacterInstances to filter the original SpeechGroup
parent_speeches = speeches.filterAddrInstances(parent_addrs)

print('Speeches in which Achilles addresses a parent:')
for s in parent_speeches:
    print(f'{s.author.name} {s.work.title} {s.l_range}')

Parent addressees of Achilles:
Thetis

Speeches in which Achilles addresses a parent:
Homer Iliad 1.352-1.356
Homer Iliad 1.365-1.412
Homer Iliad 18.79-18.93
Homer Iliad 18.98-18.126
Homer Iliad 19.21-19.27
Homer Iliad 24.139-24.140
